In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import model

# グラフの構築

In [2]:
X_input = tf.placeholder(np.uint8, shape=[None, 784])

In [3]:
image = tf.reshape(X_input, [-1, 28, 28, 1])

In [4]:
image_normed = tf.cast(image, tf.float32) / 255.0 * 2 - 1

In [5]:
with tf.variable_scope('classifier', reuse=tf.AUTO_REUSE):
    y_predicted_ = model.classifier(image_normed, classnum=12)

In [6]:
y_input = tf.placeholder(np.uint8, [None])

In [7]:
y_input_ = tf.one_hot(y_input, 12)

In [8]:
y_predicted = tf.argmax(y_predicted_, axis=1)

In [9]:
acc = tf.metrics.accuracy(    y_input,    y_predicted)

In [10]:
loss = -tf.reduce_mean(y_input_ * tf.log(y_predicted_))

In [11]:
opt = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# データのロード

In [12]:
import glob

In [13]:
X = []
y = []
for index, file in enumerate(sorted(glob.glob('data/*.npy'))):
    data = np.load(file)
    X.append(data)
    y.append(np.ones(len(data), dtype=int) * index)
    
X = np.vstack(X)
y = np.hstack(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [ ]:
from tqdm import tqdm

batch_num = 128

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver = tf.train.Saver()

for epoch in range(20):
    buf = []
    for index in tqdm(range(0, len(X_train), batch_num)):
        X_batch = X_train[index:index+batch_num]
        y_batch = y_train[index:index+batch_num]
        _, loss_train = sess.run([opt, loss], {X_input: X_batch, y_input: y_batch})
        buf.append(loss_train)
        
    loss_train = np.mean(buf)
    
    buf = []
    for index in tqdm(range(0, len(X_test), batch_num)):
        X_batch = X_test[index:index+batch_num]
        y_batch = y_test[index:index+batch_num]
        loss_test = sess.run([loss], {X_input: X_batch, y_input: y_batch})
        buf.append(loss_test)

    loss_test = np.mean(buf)
        
    print("epoch {}: loss_train: {}, loss_test: {}".format(epoch, loss_train, loss_test))
    with open('log.txt', 'a') as f:
        print("epoch {}: loss_train: {}, loss_test: {}".format(epoch, loss_train, loss_test), file=f)
    saver.save(sess, './checkpoint/model', global_step=epoch)    


        

 12%|█▏        | 1405/11716 [00:42<05:15, 32.72it/s]